<a href="https://colab.research.google.com/github/Ambrgna/CS-5530-Assignment-2/blob/main/used_cars/src/used_cars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import pandas as pd
import numpy as np

Read Raw Data

In [34]:
url = 'https://raw.githubusercontent.com/Ambrgna/CS-5530-Assignment-2/refs/heads/main/used_cars/data_raw/raw_used_cars_data.csv'
df = pd.read_csv(url)
df.head(10)

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50
5,7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,Diesel,Automatic,First,11.36 kmpl,2755 CC,171.5 bhp,8.0,21 Lakh,17.50
6,8,Volkswagen Vento Diesel Comfortline,Pune,2013,64430,Diesel,Manual,First,20.54 kmpl,1598 CC,103.6 bhp,5.0,NaN,5.20
7,9,Tata Indica Vista Quadrajet LS,Chennai,2012,65932,Diesel,Manual,Second,22.3 kmpl,1248 CC,74 bhp,5.0,NaN,1.95
8,10,Maruti Ciaz Zeta,Kochi,2018,25692,Petrol,Manual,First,21.56 kmpl,1462 CC,103.25 bhp,5.0,10.65 Lakh,9.95
9,11,Honda City 1.5 V AT Sunroof,Kolkata,2012,60000,Petrol,Automatic,First,16.8 kmpl,1497 CC,116.3 bhp,5.0,NaN,4.49


Look for the missing values

In [35]:
df.isna().sum()/len(df)*100

,0
Unnamed: 0,0.000000
Name,0.000000
Location,0.000000
Year,0.000000
Kilometers_Driven,0.000000
Fuel_Type,0.000000
Transmission,0.000000
Owner_Type,0.000000
Mileage,0.034206
Engine,0.615700


impute or drop

In [36]:
df_impute = df.copy()
df_impute.dropna(thresh=0.8*len(df_impute),axis=1,inplace=True)
df_impute.dropna(axis=0,inplace=True)
df_impute.isna().sum()

,0
Unnamed: 0,0
Name,0
Location,0
Year,0
Kilometers_Driven,0
Fuel_Type,0
Transmission,0
Owner_Type,0
Mileage,0
Engine,0


The New_Price column has more than 80% missing values, so I dropped the entire column. With that much missing data, imputing would not be meaningful and the column would not contribute reliable information.

The missing values in Mileage, Engine, Power, and Seats are each less than 1% of the dataset. Because so few rows are affected, dropping these rows does not significantly reduce the dataset or impact the analysis. Imputing such a small number of values is unnecessary.

Remove the units

In [37]:
df_numeric = df_impute.copy()
mileage_split  = df_numeric["Mileage"].str.split(" ", expand=True)
engine_split  = df_numeric["Engine"].str.split(" ", expand=True)
power_split  = df_numeric["Power"].str.split(" ", expand=True)
print(mileage_split [1].unique())
print(engine_split [1].unique())
print(power_split [1].unique())

df_numeric["Mileage"] = (
    mileage_split[0].astype(float)
    .where(mileage_split[1] == "kmpl", mileage_split[0].astype(float) * 0.54)
)
df_numeric["Engine"] = engine_split [0].astype(float)
df_numeric["Power"] = power_split [0].astype(float)
df_numeric.dtypes

['kmpl' 'km/kg']
['CC']
['bhp']


,0
Unnamed: 0,int64
Name,object
Location,object
Year,int64
Kilometers_Driven,int64
Fuel_Type,object
Transmission,object
Owner_Type,object
Mileage,float64
Engine,float64


Numerical one hot encoded

In [38]:
df_encoded = pd.get_dummies(df_numeric, columns=["Fuel_Type","Transmission"],dtype=int)
df_encoded.head(10)

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_Type_Diesel,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582.0,126.20,5.0,12.50,1,0,0,1
1,2,Honda Jazz V,Chennai,2011,46000,First,7.02,1199.0,88.70,5.0,4.50,0,1,0,1
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248.0,88.76,7.0,6.00,1,0,0,1
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968.0,140.80,5.0,17.74,1,0,1,0
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461.0,63.10,5.0,3.50,1,0,0,1
5,7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,First,11.36,2755.0,171.50,8.0,17.50,1,0,1,0
6,8,Volkswagen Vento Diesel Comfortline,Pune,2013,64430,First,20.54,1598.0,103.60,5.0,5.20,1,0,0,1
7,9,Tata Indica Vista Quadrajet LS,Chennai,2012,65932,Second,22.30,1248.0,74.00,5.0,1.95,1,0,0,1
8,10,Maruti Ciaz Zeta,Kochi,2018,25692,First,21.56,1462.0,103.25,5.0,9.95,0,1,0,1
9,11,Honda City 1.5 V AT Sunroof,Kolkata,2012,60000,First,16.80,1497.0,116.30,5.0,4.49,0,1,1,0


Create one more feature (Yearly_Millage)

In [39]:
from datetime import datetime

df_feature = df_encoded.copy()

current_year = datetime.now().year

yearly_millage = (df_feature['Kilometers_Driven'] / (current_year-df_feature["Year"])).round(2)

index = df_feature.columns.get_loc("Kilometers_Driven")
df_feature.insert(index + 1, "Yearly_Millage", yearly_millage)
df_feature

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Yearly_Millage,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_Type_Diesel,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,4100.00,First,19.67,1582.0,126.20,5.0,12.50,1,0,0,1
1,2,Honda Jazz V,Chennai,2011,46000,3285.71,First,7.02,1199.0,88.70,5.0,4.50,0,1,0,1
2,3,Maruti Ertiga VDI,Chennai,2012,87000,6692.31,First,20.77,1248.0,88.76,7.0,6.00,1,0,0,1
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,3389.17,Second,15.20,1968.0,140.80,5.0,17.74,1,0,1,0
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,7249.92,First,23.08,1461.0,63.10,5.0,3.50,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5842,6014,Maruti Swift VDI,Delhi,2014,27365,2487.73,First,28.40,1248.0,74.00,5.0,4.75,1,0,0,1
5843,6015,Hyundai Xcent 1.1 CRDi S,Jaipur,2015,100000,10000.00,First,24.40,1120.0,71.00,5.0,4.00,1,0,0,1
5844,6016,Mahindra Xylo D4 BSIV,Jaipur,2012,55000,4230.77,Second,14.00,2498.0,112.00,8.0,2.90,1,0,0,1
5845,6017,Maruti Wagon R VXI,Kolkata,2013,46000,3833.33,First,18.90,998.0,67.10,5.0,2.65,0,1,0,1


Select

In [40]:
df_selected = df_feature[["Name", "Year", "Fuel_Type_Petrol", "Price"]]
df_selected

,Name,Year,Fuel_Type_Petrol,Price
0,Hyundai Creta 1.6 CRDi SX Option,2015,0,12.50
1,Honda Jazz V,2011,1,4.50
2,Maruti Ertiga VDI,2012,0,6.00
3,Audi A4 New 2.0 TDI Multitronic,2013,0,17.74
4,Nissan Micra Diesel XV,2013,0,3.50
...,...,...,...,...
5842,Maruti Swift VDI,2014,0,4.75
5843,Hyundai Xcent 1.1 CRDi S,2015,0,4.00
5844,Mahindra Xylo D4 BSIV,2012,0,2.90
5845,Maruti Wagon R VXI,2013,1,2.65


filter

In [41]:
df_filter = df_selected[df_selected["Year"] > 2010]
df_filter

,Name,Year,Fuel_Type_Petrol,Price
0,Hyundai Creta 1.6 CRDi SX Option,2015,0,12.50
1,Honda Jazz V,2011,1,4.50
2,Maruti Ertiga VDI,2012,0,6.00
3,Audi A4 New 2.0 TDI Multitronic,2013,0,17.74
4,Nissan Micra Diesel XV,2013,0,3.50
...,...,...,...,...
5842,Maruti Swift VDI,2014,0,4.75
5843,Hyundai Xcent 1.1 CRDi S,2015,0,4.00
5844,Mahindra Xylo D4 BSIV,2012,0,2.90
5845,Maruti Wagon R VXI,2013,1,2.65


rename

In [42]:
df_renamed = df_feature.copy()
df_renamed.rename(columns={"Kilometers_Driven": "Odometer"}, inplace=True)
df_renamed

,Unnamed: 0,Name,Location,Year,Odometer,Yearly_Millage,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_Type_Diesel,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,4100.00,First,19.67,1582.0,126.20,5.0,12.50,1,0,0,1
1,2,Honda Jazz V,Chennai,2011,46000,3285.71,First,7.02,1199.0,88.70,5.0,4.50,0,1,0,1
2,3,Maruti Ertiga VDI,Chennai,2012,87000,6692.31,First,20.77,1248.0,88.76,7.0,6.00,1,0,0,1
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,3389.17,Second,15.20,1968.0,140.80,5.0,17.74,1,0,1,0
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,7249.92,First,23.08,1461.0,63.10,5.0,3.50,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5842,6014,Maruti Swift VDI,Delhi,2014,27365,2487.73,First,28.40,1248.0,74.00,5.0,4.75,1,0,0,1
5843,6015,Hyundai Xcent 1.1 CRDi S,Jaipur,2015,100000,10000.00,First,24.40,1120.0,71.00,5.0,4.00,1,0,0,1
5844,6016,Mahindra Xylo D4 BSIV,Jaipur,2012,55000,4230.77,Second,14.00,2498.0,112.00,8.0,2.90,1,0,0,1
5845,6017,Maruti Wagon R VXI,Kolkata,2013,46000,3833.33,First,18.90,998.0,67.10,5.0,2.65,0,1,0,1


mutate

In [43]:
df_mutated = df_encoded.copy()
df_mutated["Kilometers_Driven"] = (df["Kilometers_Driven"] * 0.621371).round(2)

df_mutated.rename(columns={"Kilometers_Driven": "Miles_Driven"}, inplace=True)

df_mutated

,Unnamed: 0,Name,Location,Year,Miles_Driven,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_Type_Diesel,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,25476.21,First,19.67,1582.0,126.20,5.0,12.50,1,0,0,1
1,2,Honda Jazz V,Chennai,2011,28583.07,First,7.02,1199.0,88.70,5.0,4.50,0,1,0,1
2,3,Maruti Ertiga VDI,Chennai,2012,54059.28,First,20.77,1248.0,88.76,7.0,6.00,1,0,0,1
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,25271.16,Second,15.20,1968.0,140.80,5.0,17.74,1,0,1,0
4,6,Nissan Micra Diesel XV,Jaipur,2013,54058.66,First,23.08,1461.0,63.10,5.0,3.50,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5842,6014,Maruti Swift VDI,Delhi,2014,17003.82,First,28.40,1248.0,74.00,5.0,4.75,1,0,0,1
5843,6015,Hyundai Xcent 1.1 CRDi S,Jaipur,2015,62137.10,First,24.40,1120.0,71.00,5.0,4.00,1,0,0,1
5844,6016,Mahindra Xylo D4 BSIV,Jaipur,2012,34175.40,Second,14.00,2498.0,112.00,8.0,2.90,1,0,0,1
5845,6017,Maruti Wagon R VXI,Kolkata,2013,28583.07,First,18.90,998.0,67.10,5.0,2.65,0,1,0,1


arrange

In [44]:

df_sorted = df_feature.sort_values(by="Year",ascending=False)
df_sorted

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Yearly_Millage,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_Type_Diesel,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual
1042,1077,Maruti Vitara Brezza ZDi Plus,Kochi,2019,34688,5781.33,First,24.30,1248.0,88.5,5.0,10.49,1,0,0,1
5284,5434,Honda WR-V Edge Edition i-VTEC S,Kochi,2019,13913,2318.83,First,17.50,1199.0,88.7,5.0,8.20,0,1,0,1
2639,2728,Maruti Baleno Alpha CVT,Kochi,2019,13574,2262.33,First,21.40,1197.0,83.1,5.0,9.51,0,1,1,0
5608,5769,Tata Tiago 1.2 Revotron XZ,Coimbatore,2019,20712,3452.00,First,23.84,1199.0,84.0,5.0,6.94,0,1,0,1
492,512,Hyundai Creta 1.6 SX Plus Dual Tone Petrol,Kochi,2019,8587,1431.17,First,15.29,1591.0,121.3,5.0,13.56,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185,1224,Maruti Zen VX,Jaipur,1999,70000,2692.31,Second,17.30,993.0,60.0,5.0,0.77,0,1,0,1
1791,1845,Honda City 1.3 EXI,Pune,1999,140000,5384.62,First,13.00,1343.0,90.0,5.0,0.90,0,1,0,1
3630,3749,Mercedes-Benz E-Class 250 D W 210,Mumbai,1998,55300,2048.15,First,10.00,1796.0,157.7,5.0,3.90,1,0,1,0
3039,3138,Maruti Zen LXI,Jaipur,1998,95150,3524.07,Third,17.30,993.0,60.0,5.0,0.45,0,1,0,1


summarize with group by operations

In [45]:
df_feature.groupby("Year")["Price"].mean()


,Price
Year,
1998,1.626667
1999,0.835000
2000,1.175000
2001,0.975000
2002,1.273636
2003,1.697857
2004,2.012000
2005,2.290952
2006,3.673485
